In [0]:
%sql
DESCRIBE openbet_raw.dimdate;


In [0]:
%sql
SELECT DISTINCT t.billable FROM openbet_raw.tasks t;


In [0]:
%sql
--How many unbillable hours per month

SELECT
  d.calendarmonthname,
  d.calendaryear,
  SUM(f.recordedtimehours) AS unbillable_hours
FROM openbet_operational.fact_feed f
JOIN openbet_raw.tasks t ON f.task_id = t.id
JOIN openbet_raw.dimdate d ON f.datekey = d.datekey
WHERE t.billable = "Unbillable"
  AND d.calendaryear = 2021 -- ? only available year in dataset
GROUP BY d.calendarmonthname, d.calendaryear, d.calendarmonthnumber
ORDER BY d.calendaryear, d.calendarmonthnumber

In [0]:
%sql
--Year-To-Date (YTD) recorded billable hours

WITH max_date_per_month AS (
  SELECT calendaryear, calendarmonthnumber, MAX(datekey) AS max_datekey
  FROM openbet_raw.dimdate
  GROUP BY calendaryear, calendarmonthnumber
)

SELECT
  d.calendarmonthname,
  d.calendaryear,
  c.clientname,
  SUM(f.recordedtimehours) AS ytd_billable_hours
FROM openbet_operational.fact_feed f
JOIN openbet_raw.tasks t ON f.task_id = t.id
JOIN openbet_raw.clients c ON t.client_id = c.id
JOIN openbet_raw.dimdate d ON f.datekey = d.datekey
JOIN max_date_per_month md ON d.calendaryear = md.calendaryear AND d.calendarmonthnumber = md.calendarmonthnumber
WHERE t.billable = "Billable"
  AND d.calendaryear = 2021 -- use actual year available
  AND f.datekey <= md.max_datekey
GROUP BY d.calendarmonthname, d.calendaryear, d.calendarmonthnumber, c.clientname
ORDER BY d.calendaryear, d.calendarmonthnumber;